# Week 6 Exercises

_McKinney 6.1_

There are multiple ways to solve the problems below.  You can use any one of several approaches.  For example, you can read CSV files using Pandas or the csv module.  Your score won't depend on which modules you choose to use unless explicitly noted below, but your programming style will still matter.

### 30.1 List of Allergies

In the /data directory on the Jupyter server, there is a file called `allergies.json` that contains a list of patient allergies.  It is taken from sample data provided by the EHR vendor, Epic, here: https://open.epic.com/Clinical/Allergy

Take some time to look at the structure of the file.  You can open it directly in Jupyter by clicking the _Home_ icon, then the _from_instructor_ folder, and then the _data_ folder.

Within the file, you'll see that it is a dictionary with many items in it.  One of those items is called `entry` and that item is a list of things.  You can tell that because the item name is immediately followed by an opening square bracket, signifying the start of a list.  It's line 11 of the file: `  "entry": [`

Write a function named `allergy_count(json_file)` that takes as one parameter the name of the JSON file and returns an integer number of entries in that file.  Your function should open the file, read the json into a Python object, and return how many items there are in the list of `entry`s.

In [109]:
import json
ALLERGIES_FILE="allergies.json"

In [110]:
# Put your solution here

ALLERGIES_FILE = "allergies.json"

def allergy_count(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
        return len(data.get('entry', []))



In [111]:
allergy_count(ALLERGIES_FILE)

4

In [112]:
assert type(allergy_count(ALLERGIES_FILE)) == int
assert allergy_count(ALLERGIES_FILE) == 4

### 30.2 Number of Patients

If you dig a little bit deaper into this list of allergies, you'll see that each result has a patient associated with it.  Create a funcation called `patient_count(json_file)` that will count how many unique patients we have in this JSON structure.  

In [113]:
# Put your solution here

def patient_count(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
        patients = {entry['resource']['patient']['reference'] for entry in data.get('entry', [])}
        return len(patients)

ALLERGIES_FILE = 'allergies.json'


In [114]:
patient_count(ALLERGIES_FILE)

2

In [115]:
assert type(patient_count(ALLERGIES_FILE)) == int
assert patient_count(ALLERGIES_FILE) == 2

### 30.3 How Many Allergies per Patient

Although each entry is a separate allergy, several of them are for the same patient.  Write a function called `allergy_per_patient(json_file)` that counts up how many allergies each patient has.


In [116]:
# Put your solution here
from collections import defaultdict
import json
def allergy_per_patient(json_file):

    with open(json_file, 'r') as f:
        data = json.load(f)
        allergy_counts = defaultdict(int)

        # Loop through each entry and count allergies per patient
        for entry in data.get('entry', []):
           # Extract patient name from the 'resource' section
            patient_name = entry['resource']['patient']['display']
            allergy_counts[patient_name] += 1

        return dict(allergy_counts)


In [117]:
allergy_per_patient(ALLERGIES_FILE)

{'Jason Argonaut': 3, 'Paul Boal': 1}

In [118]:
assert type(allergy_per_patient(ALLERGIES_FILE)) == dict
assert allergy_per_patient(ALLERGIES_FILE) == {'Paul Boal': 1, 'Jason Argonaut': 3}

### 30.4 Patient Allergies and Reaction

You'll see in the file that each of the items in the `entry` list have several other attributes including a patient name, substance text representation, and a reaction manifestation.  Create a function named `allergy_list(json_file)` that will create an output list that has patient name, allergy, and reaction for each `entry`.  The actual result you should get will be:

```python
[['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis']]
```

You'll notice that the reaction and the manifestation of that action are lists.  You only need to capture the first reaction and the first manifestation of the action.  That is, if there is a list of things, just output the first one.

In [119]:
# Put your solution here
import json

def allergy_list(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    allergy_summary = []

    for entry in data['entry']:
        resource = entry['resource']
        patient = resource['patient']['display']
        substance = resource['substance']['text']
        if 'reaction' in resource and resource['reaction']:
          first_reaction = resource['reaction'][0]
        if 'manifestation' in first_reaction and first_reaction['manifestation']:
            first_manifestation = first_reaction['manifestation'][0]['text']
            # Append only the patient, substance, and first manifestation as a list
            allergy_summary.append([patient, substance, first_manifestation])


    return allergy_summary

In [120]:
allergy_list(ALLERGIES_FILE)

[['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis']]

In [121]:
assert (allergy_list(ALLERGIES_FILE)) == [['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis']]

### 30.5 Allergy Reaction

Write a function called `allergy_reaction(json_file,patient,substance)` that takes three parameter and returns the reaction that will happen if the patient takes the specified substance.  You can solve this, in part, by calling your `allergy_list` function inside your new `allergy_reaction` function.

If the substance is not found in the allergy list, the function should return None.

In [122]:
# Put your solution here
import json

def allergy_list(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)  # Load the JSON data into a Python object
        result_list = []

        # Loop through each entry to create the output list
        for entry in data.get('entry', []):
            patient_name = entry['resource']['patient']['display']  # Get the patient name
            allergy = entry['resource']['substance']['text']  # Get the substance text
            # Get the first reaction and first manifestation if available
            reaction = entry['resource']['reaction'][0]['manifestation'][0]['text'] if entry['resource']['reaction'] else None

            result_list.append([patient_name, allergy, reaction])  # Append to the result list

        return result_list

def allergy_reaction(json_file, patient, substance):
    # Call the allergy_list function to get the allergy information
    allergy_data = allergy_list(json_file)

    # Search for the specified patient and substance in the allergy data
    for entry in allergy_data:
        if entry[0] == patient and entry[1] == substance:
            return entry[2]  # Return the reaction if found

    return None


In [123]:
allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN G')

'Hives'

In [124]:
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN G') == 'Hives'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS') == 'Itching'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'STRAWBERRY') == 'Anaphylaxis'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN') == None
assert allergy_reaction(ALLERGIES_FILE, 'Paul Boal', 'PENICILLIN G') == 'Bruising'

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to save this notebook file back to GitHub.  To do that in Google Colab:
1. File -> Save a Copy in GitHub
2. Make sure your HDS5210 repository is selected
3. Make sure the file name includes the week number like this: `week06/week06_assignment_2.ipynb`
4. Add a commit message that means something

**Be sure week names are lowercase and use a two digit week number!!**

**Be sure you use the same file name provided by the instructor!!**

